In [1]:
!pip install confluent_kafka sseclient

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 6.3 MB/s eta 0:00:0000:0100:01
  Created wheel for sseclient: filename=sseclient-0.0.27-py3-none-any.whl size=5563 sha256=845f524f167135245ce0b3fc4af4b702ff7b3cf7b0958cc8cf6078e10e6bca16
  Stored in directory: /home/jovyan/.cache/pip/wheels/7c/54/eb/a223b1599728ecaf0528281c17c96c503aa7d18a752a4e4e3a
Successfully built sseclient


In [1]:
from confluent_kafka import SerializingProducer, DeserializingConsumer
from confluent_kafka.serialization import StringSerializer, StringDeserializer
from confluent_kafka.admin import AdminClient, NewTopic
from uuid import uuid4
import sys, random


brokers = "kafka1:9092,kafka2:9093"
topic = "taxi_rides"

In [2]:
pconf = {
    'bootstrap.servers': brokers,
    'partitioner': 'murmur2_random',
    'key.serializer': StringSerializer('utf_8'),
    'value.serializer':  StringSerializer('utf_8')
}

In [3]:
p = SerializingProducer(pconf)

In [4]:
from datetime import datetime  
  #2023-10-13T08:16:13Z
def construct_stock(row):
    time_stamp = time.time()
    date_time = datetime.fromtimestamp(time_stamp)
    str_date_time = date_time.strftime("%Y-%m-%dT%H:%M:%SZ") #"%d-%m-%Y, %H:%M:%S"
    stock = {"TODO": row[6],
             "TODO": float(row[2]),
             "timestamp":str_date_time
             }
    return stock


def construct_taxi_ride(row):
    time_stamp = time.time()
    date_time = datetime.fromtimestamp(time_stamp)
    str_date_time = date_time.strftime("%Y-%m-%dT%H:%M:%SZ")  # ISO format timestamp
    taxi_ride = {
        "medallion": row[0],
        "hack_license": row[1],
        "pickup_datetime": row[5],
        "dropoff_datetime": row[6],
        "pickup_longitude": float(row[10]),
        "pickup_latitude": float(row[11]),
        "dropoff_longitude": float(row[12]),
        "dropoff_latitude": float(row[13]),
        "timestamp": str_date_time
    }
    return taxi_ride

In [ ]:
import csv, json
import time
n = 0
with open('sample.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    header = next(csv_reader)  # Skip the header row
    try:
        for row in csv_reader:
            #if n == 100:
             #  break
            taxi_ride = construct_taxi_ride(row)
            # Uncomment this to see sent rows
            #print(taxi_ride) 
            p.produce(topic, value=json.dumps(taxi_ride))
            p.poll(0)
            # Commented this out since it creates a 5 minute delay between sending each row
            #p.flush()
            time.sleep(0.5)
            #n = n + 1
    except BufferError:
        sys.stderr.write('%% Local producer queue is full (%d messages awaiting delivery): try again\n' % len(p))

    

### Use only if you need to purge all the messages in the queue

In [ ]:
brokers = "kafka1:9092,kafka2:9093"

admin_client = AdminClient({"bootstrap.servers":brokers})
admin_client.delete_topics(topics=["taxi_rides"])